Open Play Shots - Expected Miss Model - Model Tuning and Building - GBM

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import miss_open_shots_file_path
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Model Version

In [2]:
model_version = 7
model_name = 'expected_miss_open'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/"

In [3]:
RESPONSE = ModellingDataContract.RESPONSE_MISS

In [4]:
FEATURES = ModellingDataContract.feature_list_open_miss

In [5]:
MONOTONE_CONSTRAINTS = ModellingDataContract.monotone_constraints_open_miss

Load Data

In [6]:
df_modelling = pd.read_csv(miss_open_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,Goal,Behind,Miss,Score,Event_Type1,Set_Shot,x0,y0,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,MissTrainingSet,MissTestSet,MissValidationSet
14170,166,ballUp,behind,1260.0,3,1299,1303.0,Collingwood,Collingwood,Mason Cox,Mason_Cox,Kick,73.0,-9.0,ineffective,True,NaN,141,160,Collingwood,Brisbane Lions,right,2023F4_Collingwood_BrisbaneLions,2023F4,NaN,2023.0,0,1,0,1,Handball Received,False,73.0,-9.0,0.000000,11.401754,0.909753,0.374081,False,False,True
14171,184,possGain,behind,1430.0,4,122,156.0,Collingwood,Collingwood,Nick Daicos,Nick_Daicos,Kick,72.0,24.0,ineffective,True,NaN,141,160,Collingwood,Brisbane Lions,right,2023F4_Collingwood_BrisbaneLions,2023F4,NaN,2023.0,0,1,0,1,Gather,False,72.0,24.0,2.000000,25.298221,1.249046,0.081301,True,False,False
14172,207,possGain,behind,1587.0,4,744,746.0,Brisbane Lions,Brisbane Lions,Keidean Coleman,Keidean_Coleman,Kick,26.0,-1.0,ineffective,True,NaN,141,160,Collingwood,Brisbane Lions,right,2023F4_Collingwood_BrisbaneLions,2023F4,NaN,2023.0,0,1,0,1,Handball Received,False,26.0,-1.0,7.071068,54.009258,0.018516,0.118895,True,False,False
14173,221,ballUp,goal,1718.0,4,1109,1128.0,Brisbane Lions,Brisbane Lions,Charlie Cameron,Charlie_Cameron,Kick,73.0,6.0,effective,True,NaN,141,160,Collingwood,Brisbane Lions,right,2023F4_Collingwood_BrisbaneLions,2023F4,NaN,2023.0,1,0,0,6,Hard Ball Get,False,73.0,6.0,2.000000,9.219544,0.708626,0.599251,True,False,False
14174,222,centreBounce,goal,1732.0,4,1202,1219.0,Collingwood,Collingwood,Jordan De Goey,Jordan_De_Goey,Kick,32.0,8.0,effective,True,NaN,141,160,Collingwood,Brisbane Lions,right,2023F4_Collingwood_BrisbaneLions,2023F4,NaN,2023.0,1,0,0,6,Handball Received,False,32.0,8.0,7.211103,48.662100,0.165149,0.130293,True,False,False


In [7]:
df_modelling[['ballUp', 'centreBounce', 'kickIn', 'possGain', 'throwIn']] = pd.get_dummies(df_modelling['Initial_State'])

In [8]:
training_data = df_modelling[df_modelling[RESPONSE+"TrainingSet"]]
test_data = df_modelling[df_modelling[RESPONSE+"TestSet"]]
cal_data = df_modelling[df_modelling[RESPONSE+"ValidationSet"]]

In [9]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]
X_cal, y_cal = cal_data.drop(columns=[RESPONSE]), cal_data[RESPONSE]

In [10]:
y_train.mean(), y_test.mean(), y_cal.mean()

(0.207010582010582, 0.21587301587301588, 0.22001763668430335)

In [11]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,Goal,Behind,Score,Event_Type1,Set_Shot,x0,y0,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,MissTrainingSet,MissTestSet,MissValidationSet,ballUp,centreBounce,kickIn,possGain,throwIn
1,6,possGain,goal,61.0,1,149,168.0,Brisbane Lions,Brisbane Lions,Zac Bailey,Zac_Bailey,Kick,35.0,19.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,1,0,6,Handball Received,False,35.0,19.0,7.000000,47.010637,0.416065,0.125105,False,True,False,False,False,False,True,False
5,43,possGain,behind,444.0,1,1272,1317.0,Brisbane Lions,Brisbane Lions,Joe Daniher,Joe_Daniher,Kick,27.0,31.0,ineffective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,0,1,1,Handball Received,False,27.0,31.0,3.162278,59.682493,0.546167,0.091898,False,True,False,False,False,False,True,False
9,83,throwIn,goal,781.0,2,848,861.0,Brisbane Lions,Brisbane Lions,Jarryd Lyons,Jarryd_Lyons,Kick,46.0,2.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,1,0,6,Gather,False,46.0,2.0,5.000000,32.062439,0.062419,0.201226,False,True,False,False,False,False,False,True
11,91,possGain,miss,863.0,2,1223,1226.0,Brisbane Lions,Brisbane Lions,Zac Bailey,Zac_Bailey,Kick,54.0,-39.0,clanger,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,0,0,0,Hard Ball Get Crumb,False,54.0,-39.0,7.810250,45.793013,1.019141,0.073607,False,True,False,False,False,False,True,False
19,161,possGain,goal,1422.0,3,1570,1575.0,Brisbane Lions,Brisbane Lions,Jarrod Berry,Jarrod_Berry,Kick,38.0,-37.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,1,0,6,Handball Received,False,38.0,-37.0,9.219544,54.488531,0.746457,0.086523,False,True,False,False,False,False,True,False


Preprocess Data

In [12]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [13]:
X_train_preproc = X_train[FEATURES]
X_test_preproc = X_test[FEATURES]
X_cal_preproc = X_cal[FEATURES]

In [14]:
X_train_preproc.shape, X_test_preproc.shape

((9072, 6), (2835, 6))

In [15]:
X_train_preproc.head()

,x0,y0,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle
0,26.0,-21.0,8.246211,56.080300,0.383818,0.106164
2,50.0,24.0,4.000000,36.878178,0.708626,0.132764
3,34.0,15.0,15.132746,46.486557,0.328553,0.130930
4,40.0,-11.0,4.123106,39.560081,0.281772,0.156423
6,56.0,6.0,15.132746,22.803509,0.266252,0.276208


In [16]:
X_test_preproc.head()

,x0,y0,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle
1,35.0,19.0,7.000000,47.010637,0.416065,0.125105
5,27.0,31.0,3.162278,59.682493,0.546167,0.091898
9,46.0,2.0,5.000000,32.062439,0.062419,0.201226
11,54.0,-39.0,7.810250,45.793013,1.019141,0.073607
19,38.0,-37.0,9.219544,54.488531,0.746457,0.086523


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [17]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints=MONOTONE_CONSTRAINTS)

In [18]:
xgb_tuner.training_data.head()

,x0,y0,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle
0,26.0,-21.0,8.246211,56.080300,0.383818,0.106164
2,50.0,24.0,4.000000,36.878178,0.708626,0.132764
3,34.0,15.0,15.132746,46.486557,0.328553,0.130930
4,40.0,-11.0,4.123106,39.560081,0.281772,0.156423
6,56.0,6.0,15.132746,22.803509,0.266252,0.276208


In [19]:
xgb_tuner.tune_hyperparameters()

[I 2023-10-21 14:33:02,765] A new study created in memory with name: no-name-8e48437f-0880-4b85-aaea-1a5b1770e056
[I 2023-10-21 14:33:02,807] Trial 0 finished with value: 0.5101803457885568 and parameters: {'max_depth': 2, 'min_child_weight': 75, 'eta': 0.7621561213779372, 'gamma': 0.1155003204825153, 'lambda': 0.0002775331422104276, 'alpha': 0.0011832236140806752, 'subsample': 0.6401976767867414, 'colsample_bytree': 0.4864271478316756}. Best is trial 0 with value: 0.5101803457885568.
[I 2023-10-21 14:33:02,844] Trial 1 finished with value: 0.6501717329237154 and parameters: {'max_depth': 7, 'min_child_weight': 60, 'eta': 0.013879826941622224, 'gamma': 0.37220558574037915, 'lambda': 0.00025148347170989536, 'alpha': 0.0005571023815191254, 'subsample': 0.3325031608158169, 'colsample_bytree': 0.3902762558469476}. Best is trial 0 with value: 0.5101803457885568.
[I 2023-10-21 14:33:02,900] Trial 2 finished with value: 0.4865954103719454 and parameters: {'max_depth': 12, 'min_child_weight': 

Number of finished trials:  1000
Best trial:
  Value: 0.45269481281568646
  Params: 
    max_depth: 2
    min_child_weight: 46
    eta: 0.4983796472983455
    gamma: 0.006199468174936356
    lambda: 0.006385885587509747
    alpha: 0.008323373902214621
    subsample: 0.4486535072188763
    colsample_bytree: 0.7509468204146142


In [20]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 2,
 'min_child_weight': 46,
 'eta': 0.4983796472983455,
 'gamma': 0.006199468174936356,
 'lambda': 0.006385885587509747,
 'alpha': 0.008323373902214621,
 'subsample': 0.4486535072188763,
 'colsample_bytree': 0.7509468204146142}

Training Model - SuperXGBClassifier class for training and predictions

In [21]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = MONOTONE_CONSTRAINTS

In [22]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test,
                               X_cal = X_cal_preproc,
                               y_cal = y_cal,
                               params = params)

In [23]:
super_xgb.fit()

[0]	validation_0-logloss:0.55619	validation_1-logloss:0.56185
[1]	validation_0-logloss:0.51470	validation_1-logloss:0.52309
[2]	validation_0-logloss:0.49713	validation_1-logloss:0.50800
[3]	validation_0-logloss:0.49089	validation_1-logloss:0.50151
[4]	validation_0-logloss:0.48741	validation_1-logloss:0.49790
[5]	validation_0-logloss:0.48593	validation_1-logloss:0.49670
[6]	validation_0-logloss:0.48530	validation_1-logloss:0.49590
[7]	validation_0-logloss:0.48500	validation_1-logloss:0.49641
[8]	validation_0-logloss:0.48467	validation_1-logloss:0.49614
[9]	validation_0-logloss:0.48473	validation_1-logloss:0.49681
[10]	validation_0-logloss:0.48454	validation_1-logloss:0.49554
[11]	validation_0-logloss:0.48389	validation_1-logloss:0.49437
[12]	validation_0-logloss:0.48365	validation_1-logloss:0.49397
[13]	validation_0-logloss:0.48326	validation_1-logloss:0.49416
[14]	validation_0-logloss:0.48319	validation_1-logloss:0.49397
[15]	validation_0-logloss:0.48316	validation_1-logloss:0.49430
[1

In [24]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7509468204146142, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.006199468174936356, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.498...
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=46, missing=nan,
              monotone_constraints={'Angle_to_Middle_Goal': 1,
                                    'Distance_Since_Last_Action': -1,
                                    'Distance_to_Middle_Goal': 1,
                                    'Visible_Goal_Angle': -1, 'x0': 1},
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [25]:
super_xgb.xgb_model.get_booster().feature_names

['x0',
 'y0',
 'Distance_Since_Last_Action',
 'Distance_to_Middle_Goal',
 'Angle_to_Middle_Goal',
 'Visible_Goal_Angle']

In [26]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [27]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]
cal_probas = super_xgb.predict_proba(X_cal_preproc)[:, 1]

In [28]:
super_xgb.calibrate()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [29]:
train_cal_probas = super_xgb.predict_proba(X_train_preproc, calibrate=True)
test_cal_probas = super_xgb.predict_proba(X_test_preproc, calibrate=True)

Check Average Predictions

In [30]:
train_probas.mean(), training_data[RESPONSE].mean(), train_cal_probas.mean()

(0.20520388, 0.207010582010582, 0.2220746786289033)

In [31]:
test_probas.mean(), test_data[RESPONSE].mean(), test_cal_probas.mean()

(0.20730212, 0.21587301587301588, 0.22402937535092826)

Export model

In [32]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [33]:
train_info = training_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export['xgb_probas_cal'] = train_cal_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,Match_ID,Chain_Number,Order,Miss,x0,y0,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,1,11.0,0,26.0,-21.0,8.246211,56.080300,0.383818,0.106164,0,0.133719,0.155189
1,202101_BrisbaneLions_Sydney,10,110.0,0,50.0,24.0,4.000000,36.878178,0.708626,0.132764,0,0.343976,0.354628
2,202101_BrisbaneLions_Sydney,21,222.0,0,34.0,15.0,15.132746,46.486557,0.328553,0.130930,0,0.163361,0.186880
3,202101_BrisbaneLions_Sydney,38,392.0,1,40.0,-11.0,4.123106,39.560081,0.281772,0.156423,0,0.268525,0.289442
4,202101_BrisbaneLions_Sydney,46,476.0,0,56.0,6.0,15.132746,22.803509,0.266252,0.276208,0,0.064236,0.076006


In [34]:
test_info = test_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export['xgb_probas_cal'] = test_cal_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,Match_ID,Chain_Number,Order,Miss,x0,y0,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,6,61.0,0,35.0,19.0,7.000000,47.010637,0.416065,0.125105,0,0.194604,0.218915
1,202101_BrisbaneLions_Sydney,43,444.0,0,27.0,31.0,3.162278,59.682493,0.546167,0.091898,0,0.292775,0.311089
2,202101_BrisbaneLions_Sydney,83,781.0,0,46.0,2.0,5.000000,32.062439,0.062419,0.201226,0,0.180896,0.205029
3,202101_BrisbaneLions_Sydney,91,863.0,1,54.0,-39.0,7.810250,45.793013,1.019141,0.073607,0,0.329215,0.342363
4,202101_BrisbaneLions_Sydney,161,1422.0,0,38.0,-37.0,9.219544,54.488531,0.746457,0.086523,0,0.266420,0.287531
